<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Kevin Pinochet Hernández
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/Keviinplz/MDS7202-1`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

In [2]:
!pip install numpy pandas sklearn

  Using cached pandas-1.3.4-cp39-cp39-win_amd64.whl (10.2 MB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=1c1d85f74edfd57aba6b39ec07ed1f6a51094aed48ed8515e5e1552d1a4cd3f5
  Stored in directory: c:\users\kevii\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


You should consider upgrading via the 'C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\Scripts\python.exe -m pip install --upgrade pip' command.


#Importamos librerias utiles 😸

In [3]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


You should consider upgrading via the 'C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\Scripts\python.exe -m pip install --upgrade pip' command.


  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=4818399b49c384a4b3ef7ce7b68f52070dd6cac26600d45d9d032d6b8ca6e0de
  Stored in directory: c:\users\kevii\appdata\local\pip\cache\wheels\4a\66\81\205d77bd58539a1d07fb8b795fcea0b17f4d59912c0f4202b7
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=afb386a1eb0109d7857ef24bba84cecf712b8b456a5de1588dc918033d623720
  Stored in directory: c:\users\kevii\appdata\local\pip\cache\wheels\47\8c\05\65ea39bbebd7fa341790e50ae16e50b17fa091021e54f42b84
Successfully built umap-learn pynndescent
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\Kevii\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\lab7-_Uj2crqR-py3.9\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Check the permissions.

You should consider upgrading via the 'C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\Scripts\python.exe -m pip install --upgrade pip' command.


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [8]:
import os

# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except ModuleNotFoundError: 
    print('Ignorando conexión drive-colab')
    path = os.path.join(os.getcwd(), 'data')
    
filepath = lambda filename: os.path.join(path, filename)

Ignorando conexión drive-colab


In [9]:
df_players = pd.read_csv(filepath('stats_players.csv'))
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [28]:
from IPython.display import display

labels: np.ndarray = df_players["National_Position"].unique()
print(f'Existen {len(labels)} distintas etiquetas (jugadores seleccionados):\n{labels}')

print("Agrupando por National_Position podemos conocer la cantidad de ejemplos por cada label")


display(df_players.groupby("National_Position").count()["Name"])

print(f'Los nulos de esa columna (no seleccionados) son: {df_players["National_Position"].isna().sum()}')

el_classified = df_players.groupby("National_Position").count()["Name"].sum()
el_no_classified = df_players["National_Position"].isna().sum()

# Comprobamos que no falten datos, al sumar los elementos etiquetados con los no etiquetados
assert el_classified + el_no_classified == len(df_players)

Existen 28 distintas etiquetas (jugadores seleccionados):
['LS' 'RW' 'LW' 'GK' 'RS' nan 'RCB' 'LF' 'CAM' 'Sub' 'LCB' 'RCM' 'LDM'
 'LCM' 'LAM' 'CB' 'CDM' 'LM' 'RM' 'LB' 'ST' 'CM' 'RF' 'RB' 'RDM' 'RAM'
 'LWB' 'RWB']
Agrupando por National_Position podemos conocer la cantidad de ejemplos por cada label


National_Position
CAM     19
CB       9
CDM      9
CM       9
GK      47
LAM      4
LB      39
LCB     48
LCM     25
LDM     19
LF       3
LM      32
LS      18
LW       7
LWB      4
RAM      4
RB      38
RCB     46
RCM     25
RDM     18
RF       3
RM      34
RS      18
RW       7
RWB      4
ST      30
Sub    556
Name: Name, dtype: int64

Los nulos de esa columna (no seleccionados) son: 16513


### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [80]:
numeric = df_players.select_dtypes(include=np.number).columns.tolist()
categorical_numeric = ["Weak_foot", "Skill_Moves"]

for field in categorical_numeric:
    numeric.remove(field)
    
preprocess = ColumnTransformer(
    transformers=[
        ("minmax", MinMaxScaler(), numeric),
        ("scale", StandardScaler(), numeric),
        ("onehot", OneHotEncoder(handle_unknown='ignore'), ["Nationality", "Club_Position", "Preffered_Foot", "Work_Rate"] + categorical_numeric)
])

# Hacemos un escalado min max y un escalado estandar
# esto es debido a que los datos entre si viven en dominios diferentes
# por lo que buscamos llevarlos todos al mismo dominio

# a su vez, hacemos one hot encoder para transladar al espacio numerico los labels con los que trabajamos
# hay variables categoricas numéricas, por lo que para que todo quede en el mismo dominio, entonces también
# aplicamos onehotencoder sobre ellos.

In [81]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

seed = 42

linearsvc = Pipeline([
    ("preprocess", preprocess),
    ("linear-svc", LinearSVC(random_state=seed))
])

kneighbors = Pipeline([
    ("preprocess", preprocess),
    ("k-neighbors", KNeighborsClassifier())
])

random_forest = Pipeline([
    ("preprocess", preprocess),
    ("random-forest", RandomForestClassifier(random_state=seed))
])


### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [89]:
df_players.National_Position = df_players.National_Position.fillna('NOSEC')
from sklearn.preprocessing import LabelEncoder

X, y = df_players.iloc[:, lambda x: [i for i in range(1, len(df_players.columns)) if i != 2]], df_players.iloc[:, 2].to_numpy().reshape(-1, 1)


enc = LabelEncoder()
enc.fit(y)
y = enc.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=0.2)

pipes = [
    ("linear svc", linearsvc),
    ("kneighbors", kneighbors),
    ("random_forest", random_forest)
]

for _, pipe in pipes:
    pipe.fit(X_train, y_train)

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [91]:
for pipename, pipe in pipes:
    print(f"Metricas para {pipename}:\n{classification_report(y_test, pipe.predict(X_test))}")

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

Metricas para linear svc:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         9
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00         4
          15       0.96      0.99      0.98      3324
          17       0.00      0.00      0.00         7
          18       0.25      0.11      0.15         9
          19       0.00      0.00      0.00         6
          20       0.00      0.00      0.00         7
 

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

**Respuesta**:

```
Se puede notar una clara tendencia a clasificar a no seleccionados, esto es debido a la inmensa mayoría de ejemplos con esta etiqueta, se debería entonces hacer un mejor split para balancear las clases, de tal forma de lograr una mejor clasificación. Se puede conseguir buscando más ejemplos por cada clase.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [122]:
### Código aquí ###
df_players_clean = df_players.drop(df_players[(df_players.Club_Position.str.contains("Sub") | df_players.Club_Position.str.contains("Res"))].index)

def apply_club_position_label(row):
    transformed = {
        "ST": "ataque",
        "CF": "ataque",
        "RW": "central_ataque",
        "CAM": "central_ataque",
        "LW": "central_ataque",
        "RM": "central",
        "CM": "central",
        "LM": "central",
        "RWB": "central_defensa",
        "CDM": "central_defensa",
        "LWB": "central_defensa",
        "RB": "defensa",
        "CB": "defensa",
        "LB": "defensa",
        "GK": "arquero",
    }
    return transformed.get(row, None)

df_players_clean.Club_Position = df_players_clean.Club_Position.apply(apply_club_position_label)
df_players_clean = df_players_clean.dropna()
display(df_players_clean.groupby("Club_Position").count())

,Name,Nationality,National_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
Club_Position,,,,,,,,,,,,,,,,,,,,,
arquero,632,632,632,632,632,632,632,632,632,632,...,632,632,632,632,632,632,632,632,632,632
ataque,430,430,430,430,430,430,430,430,430,430,...,430,430,430,430,430,430,430,430,430,430
central,907,907,907,907,907,907,907,907,907,907,...,907,907,907,907,907,907,907,907,907,907
central_ataque,581,581,581,581,581,581,581,581,581,581,...,581,581,581,581,581,581,581,581,581,581
central_defensa,209,209,209,209,209,209,209,209,209,209,...,209,209,209,209,209,209,209,209,209,209
defensa,1180,1180,1180,1180,1180,1180,1180,1180,1180,1180,...,1180,1180,1180,1180,1180,1180,1180,1180,1180,1180


In [126]:
numeric = df_players_clean.select_dtypes(include=np.number).columns.tolist()
categorical_numeric = ["Weak_foot", "Skill_Moves"]

for field in categorical_numeric:
    numeric.remove(field)
    
preprocess = ColumnTransformer(
    transformers=[
        ("minmax", MinMaxScaler(), numeric),
        ("scale", StandardScaler(), numeric),
        ("onehot", OneHotEncoder(handle_unknown='ignore'), ["Preffered_Foot", "Work_Rate"] + categorical_numeric)
])

linearsvc = Pipeline([
    ("preprocess", preprocess),
    ("linear-svc", LinearSVC(random_state=seed))
])

kneighbors = Pipeline([
    ("preprocess", preprocess),
    ("k-neighbors", KNeighborsClassifier())
])

random_forest = Pipeline([
    ("preprocess", preprocess),
    ("random-forest", RandomForestClassifier(random_state=seed))
])

exclude = [1, 2, 3]
X, y = df_players_clean.iloc[:, lambda x: [i for i in range(1, len(df_players_clean.columns)) if i not in exclude]], df_players_clean.iloc[:, 3].to_numpy().reshape(-1, 1)


enc = LabelEncoder()
enc.fit(y)
y = enc.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=0.2)

pipes = [
    ("linear svc", linearsvc),
    ("kneighbors", kneighbors),
    ("random_forest", random_forest)
]

for _, pipe in pipes:
    pipe.fit(X_train, y_train)

C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kevii\AppData\Local\pypoetry\Cache\virtualenvs\lab7-_Uj2crqR-py3.9\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [127]:
for pipename, pipe in pipes:
    print(f"Metricas para {pipename}:\n{classification_report(y_test, pipe.predict(X_test))}")

Metricas para linear svc:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       125
           1       0.77      0.86      0.81        86
           2       0.62      0.65      0.63       192
           3       0.63      0.36      0.46       114
           4       0.53      0.20      0.29        41
           5       0.80      0.99      0.89       230

    accuracy                           0.76       788
   macro avg       0.73      0.68      0.68       788
weighted avg       0.75      0.76      0.74       788

Metricas para kneighbors:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       125
           1       0.69      0.79      0.74        86
           2       0.54      0.63      0.58       192
           3       0.47      0.24      0.31       114
           4       0.47      0.22      0.30        41
           5       0.82      0.94      0.87       230

    accuracy             

**Respuesta**:

```
Ahora hay resultados muchos más coherentes que dan cuenta de la posibilidad de clasificar jugadores para predecir la posición del club, en este caso particular, el mejor clasificador es el Linear SVC.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [129]:
### Código aquí ###
# Por falta de tiempo, entrego hasta aquí uwu
df_sueldos = pd.read_csv(filepath("salarios.csv"))
df_sueldos.head()

# R^2 compara nuestro modelo con un baseline, y siempre es menor a 1, un R^2 cercano a 1 implica que nuestro modelo
# se ajusta bien a nuestros datos.



,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0


**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>